In [2]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv('../datasets/recdemo.csv')

In [5]:
df

,id,A,B,C,D,E,F
0,1,3.0,4.0,2.0,4.0,1.0,NaN
1,2,3.0,4.0,2.0,4.0,NaN,2.0
2,3,NaN,2.0,5.0,5.0,NaN,5.0
3,4,NaN,NaN,NaN,NaN,4.0,NaN
4,5,3.0,NaN,2.0,4.0,4.0,NaN
5,6,NaN,5.0,5.0,5.0,NaN,5.0
6,7,1.0,NaN,NaN,2.0,NaN,3.0
7,8,NaN,NaN,NaN,NaN,4.0,4.0


In [6]:
df_unpivot=pd.melt(df, id_vars=['id'])
df_unpivot.head()

,id,variable,value
0,1,A,3.0
1,2,A,3.0
2,3,A,NaN
3,4,A,NaN
4,5,A,3.0


In [7]:
df_unpivot.dropna(inplace=True)
df_unpivot.columns = ['userID', 'itemID', 'rating']

In [8]:
from surprise import Dataset
from surprise import Reader

In [9]:
reader = Reader(rating_scale=(1, 5)) # Зададим разброс оценок
data = Dataset.load_from_df(df_unpivot[['userID', 'itemID', 'rating']], reader)

In [10]:
trainset = data.build_full_trainset()
testset = trainset.build_anti_testset()

In [11]:
from surprise import SVD

In [12]:
algo = SVD(n_factors=2, random_state=999)
predictions = algo.fit(trainset).test(testset)

In [17]:
df_unpivot1 = df_unpivot.copy()
for i in predictions:
    df_unpivot1 = df_unpivot1.append({'userID': i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)

/var/folders/mm/2c6bnjk559g2btv241j7f9lh0000gn/T/ipykernel_1263/878506005.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_unpivot1 = df_unpivot1.append({'userID': i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)
/var/folders/mm/2c6bnjk559g2btv241j7f9lh0000gn/T/ipykernel_1263/878506005.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_unpivot1 = df_unpivot1.append({'userID': i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)
/var/folders/mm/2c6bnjk559g2btv241j7f9lh0000gn/T/ipykernel_1263/878506005.py:3: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df_unpivot1 = df_unpivot1.append({'userID': i.uid, 'itemID': i.iid, 'rating': i.est}, ignore_index=True)
/var/folders/mm/2c6bnjk559g2btv241j7f9lh

In [18]:
df_unpivot1.pivot(index='userID', columns='itemID', values='rating')

itemID,A,B,C,D,E,F
userID,,,,,,
1,3.000000,4.000000,2.000000,4.000000,1.000000,3.312456
2,3.000000,4.000000,2.000000,4.000000,3.210599,2.000000
3,3.444350,2.000000,5.000000,5.000000,3.684146,5.000000
4,3.242607,3.599635,3.415230,3.784069,4.000000,3.647434
5,3.000000,3.490772,2.000000,4.000000,4.000000,3.533499
6,3.676756,5.000000,5.000000,5.000000,3.884204,5.000000
7,1.000000,3.194893,2.985395,2.000000,3.010203,3.000000
8,3.301363,3.630884,3.442314,3.798689,4.000000,4.000000


In [19]:
algo.pu # users

array([[-0.02356038,  0.086795  ],
       [ 0.03311668, -0.06586878],
       [-0.01663012, -0.05196951],
       [ 0.17326108, -0.22112997],
       [ 0.11922751,  0.0574848 ],
       [-0.03847042, -0.02430361],
       [-0.10099924, -0.04709509],
       [-0.00974106,  0.08383197]])

In [20]:
algo.qi # films

array([[-0.0499725 ,  0.20377982],
       [ 0.07260589, -0.09175883],
       [-0.0048234 , -0.06959711],
       [-0.08501758, -0.10890099],
       [ 0.16616968,  0.1825492 ],
       [ 0.00912318, -0.15003955]])

In [21]:
algo.qi.T # films

array([[-0.0499725 ,  0.07260589, -0.0048234 , -0.08501758,  0.16616968,
         0.00912318],
       [ 0.20377982, -0.09175883, -0.06959711, -0.10890099,  0.1825492 ,
        -0.15003955]])

In [22]:
mu = df_unpivot.rating.mean()

In [23]:
m = algo.pu @ algo.qi.T

In [24]:
m.shape

(8, 6)

In [25]:
i = 0
j = 5

In [26]:
mu + m[i][j] + algo.bu[i] + algo.bi[j]

3.31245613193024

# Сравнение параметров

In [27]:
from surprise import KNNBaseline
from surprise.model_selection import cross_validate

In [29]:
data = Dataset.load_builtin('ml-100k')

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from https://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to /Users/adwiz/.surprise_data/ml-100k


In [30]:
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user', 'item', 'rating', 'timestamp']
df.head()

,user,item,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [31]:
for i in [1, 3, 5, 7, 20]:
    algo = KNNBaseline(k=i, verbose=False)
    cv = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=False)
    print(str(i)+'NN:', np.mean(cv['test_rmse']))

1NN: 1.225571815333017
3NN: 1.0285238044123872
5NN: 0.9866453562005223
7NN: 0.9648647073917965
20NN: 0.9338949853200551


In [32]:
for i in [1, 2, 3, 5, 7, 10, 50, 70]:
    algo = SVD(n_factors=i, random_state=999, verbose=False)
    cv = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=False)
    print(str(i)+'-factors:', np.mean(cv['test_rmse']))

1-factors: 0.9408919316001134
2-factors: 0.9379691195765535
3-factors: 0.9384928479550151
5-factors: 0.9407706865677387
7-factors: 0.9365736896522062
10-factors: 0.9364465716002996
50-factors: 0.9346631962701928
70-factors: 0.9343155526908185


In [33]:
 for i in [1, 2, 3, 5, 7, 10, 50, 70]:
    algo = SVD(n_factors=i, biased=False, random_state=999, verbose=False)
    cv = cross_validate(algo, data, measures=['RMSE'], cv=5, verbose=False)
    print(str(i)+'-factors:', np.mean(cv['test_rmse']))

1-factors: 0.9595235192411314
2-factors: 0.9545346572841409
3-factors: 0.9518423842302071
5-factors: 0.9442465254329203
7-factors: 0.9521461090859145
10-factors: 0.9437824863473473
50-factors: 0.9437420832577376
70-factors: 0.9455068383093881
